<a href="https://colab.research.google.com/github/AbdalrhmanFouadOsama2004/0/blob/main/Flower_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
(ds_train, ds_val), ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

NUM_CLASSES = ds_info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tf_flowers/incomplete.SLX3FQ_3.0.1/tf_flowers-train.tfrecord*...:   0%|   …

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

BATCH_SIZE = 32

ds_train = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=5
)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.7027 - loss: 0.7813 - val_accuracy: 0.8760 - val_loss: 0.3690
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.9173 - loss: 0.2483 - val_accuracy: 0.8760 - val_loss: 0.3259
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.9465 - loss: 0.1701 - val_accuracy: 0.8924 - val_loss: 0.3073
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.9629 - loss: 0.1201 - val_accuracy: 0.9114 - val_loss: 0.2887
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.9907 - loss: 0.0612 - val_accuracy: 0.9114 - val_loss: 0.2873


In [ ]:
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history_finetune = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=5
)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.8464 - loss: 0.4109 - val_accuracy: 0.9046 - val_loss: 0.2883
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.9388 - loss: 0.1875 - val_accuracy: 0.9128 - val_loss: 0.2819
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.9472 - loss: 0.1595 - val_accuracy: 0.9114 - val_loss: 0.2765
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.9655 - loss: 0.1168 - val_accuracy: 0.9142 - val_loss: 0.2719
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.9768 - loss: 0.0949 - val_accuracy: 0.9223 - val_loss: 0.2643


In [ ]:
model.save("flower_mobilenetv2_finetuned.h5")

print("Model training and fine-tuning complete. Model saved as 'flower_mobilenetv2_finetuned.h5'")

Model training and fine-tuning complete. Model saved as 'flower_mobilenetv2_finetuned.h5'
